# Brain Tumor Detection Using DL

The main purpose of this project is detecting tumors from MRI scans. 

source: [kaggle](https://www.kaggle.com/datasets/navoneel/brain-mri-images-for-brain-tumor-detection)

### Import necessary modules

In [10]:
import os
import cv2
import shutil

from tqdm import tqdm
from PIL import Image

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt



## Splitting the data

In [4]:
img_dir = '../../Datasets/brain_tumor_dataset/'
img_dir_data = '../../Datasets/brain_tumor_dataset/data/'
for category in os.listdir(img_dir_data):
    if not category.startswith('.'):
        img_num = len(os.listdir(img_dir_data+category))
        for n, img_name in enumerate(os.listdir(img_dir_data+category)):
            img = img_dir_data + category + '/' + img_name
            if n < 5:
                shutil.copy(img, 'TEST/' + category.upper() + '/' + img_name)
            elif n < 0.8*img_num:
                shutil.copy(img, 'TRAIN/' + category.upper() + '/' + img_name)
            else:
                shutil.copy(img, 'VAL/' + category.upper() + '/' + img_name)

## Data Import and Preprocessing

In [57]:
def load_data(dir_path, img_size=(100,100)):
    """ Load Images as np.arrays
    :params:
    :return:
    """
    X = []
    y = []
    i = 0
    labels = dict()
    for path in tqdm(os.listdir(dir_path)):
        if not path.startswith('.'):
            labels[i] = path
            for file in os.listdir(dir_path + path):
                if not file.startswith('.'):
                    img = cv2.imread(dir_path+path+'/'+file)
                    img = cv2.resize(img, dsize=img_size, interpolation=cv2.INTER_CUBIC)
                    X.append(img)
                    y.append(i)
            i += 1
    X = np.array(X)
    y = np.array(y)
    print(f'{len(X)} image loaded from {dir_path} directory.')
    return X, y, labels

In [58]:
TRAIN_DIR = 'TRAIN/'
TEST_DIR = 'TEST/'
VAL_DIR = 'VAL/'
IMG_SIZE = (224,224)

X_train, y_train, labels = load_data(TRAIN_DIR, IMG_SIZE)
X_test, y_test, _ = load_data(TEST_DIR, IMG_SIZE)
X_val, y_val, _ = load_data(VAL_DIR, IMG_SIZE)

100%|██████████| 3/3 [00:00<00:00, 33.78it/s]

193 image loaded from TRAIN/ directory.
10 image loaded from TEST/ directory.
50 image loaded from VAL/ directory.


In [72]:
train_dist = np.array(np.unique(y_train, return_counts = True)).T
test_dist = np.array(np.unique(y_test, return_counts = True)).T
val_dist = np.array(np.unique(y_val, return_counts = True)).T

In [110]:
data_dist = pd.DataFrame({'Train':train_dist[:,1],'Val':val_dist[:,1], 'Test':test_dist[:,1]})
data_dist

,Train,Val,Test
0,74,19,5
1,119,31,5
